# 2. Model Calibration and Testing

In [75]:
from model.binomial import EuropeanVanillaModel
import pandas as pd
import numpy as np

#### GS Stock Prices (Nov 30, 2016 - Feb 27, 2017)

We chose these dates because they are between dividend paying dates of *Nov 29,2016* and *Feb 28, 2017*

In [148]:
stock_prices = pd.read_csv('dataset/gs-stock-prices.csv', index_col=0, parse_dates=True).iloc[::-1]

In [150]:
stock_prices['20161130':'20170227']

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-11-30,215.199997,220.770004,214.970001,219.289993,6507900,218.718307
2016-12-01,220.929993,227.160004,220.350006,226.630005,7606500,226.039183
2016-12-02,226.000000,226.250000,221.220001,223.360001,5297400,222.777704
2016-12-05,225.520004,229.199997,225.240005,228.550003,4599300,227.954176
2016-12-06,230.179993,232.669998,227.899994,231.380005,5259300,230.776800
2016-12-07,230.399994,236.089996,230.399994,235.559998,6595200,234.945895
2016-12-08,236.880005,242.419998,236.000000,241.449997,7343700,240.820540
2016-12-09,240.610001,242.000000,238.880005,241.850006,5456000,241.219506
2016-12-12,240.500000,242.830002,236.369995,237.169998,5022000,236.551699
